In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
### Preprocess the data

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)


In [4]:
# Encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])



In [6]:
# One-Hot encode 'Geography'ArithmeticError
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
# Combine one-hot-encoded columns with original data

data = pd.concat([data.drop('Geography', axis= 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Split the data into features and target

x = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [9]:
# Split data into trainijng and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
# Scale these features

scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [11]:
# save the encoders and scaler for later use

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [12]:
# ANN Regression problem statement

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [14]:
# Build the model

model = Sequential([
    Dense(64, activation = 'relu', input_shape = (xtrain.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1) # Output layer for regression
])

# Compile the model

model.compile(optimizer='adam', loss = 'mean_absolute_error', metrics = ['mae'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# set up tensorboard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [16]:
# Set up Early Staopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [17]:
# Train the model

history = model.fit(
    xtrain, ytrain,
    validation_data = (xtest, ytest),
    epochs = 100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 3s 6ms/step - loss: 100376.0156 - mae: 100376.0156 - val_loss: 98513.8672 - val_mae: 98513.8672
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99614.4141 - mae: 99614.4141 - val_loss: 96971.5391 - val_mae: 96971.5391
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 96911.8672 - mae: 96911.8672 - val_loss: 93018.5234 - val_mae: 93018.5234
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 91599.2500 - mae: 91599.2500 - val_loss: 86439.9609 - val_mae: 86439.9609
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 83867.3047 - mae: 83867.3047 - val_loss: 77977.7422 - val_mae: 77977.7422
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 74804.7656 - mae: 74804.7656 - val_loss: 69123.9219 - val_mae: 69123.9219
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 66086.7500 - mae: 66086.7500 

In [18]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 5780), started 0:00:16 ago. (Use '!kill 5780' to kill it.)

In [22]:
## Evaluate model on test data

test_loss, test_mae = model.evaluate(xtest, ytest)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 4ms/step - loss: 50272.3438 - mae: 50272.3438
Test MAE : 50272.34375


In [23]:
model.save('regression_model.h5')